In [1]:
import numpy as np 
import pandas as pd
import os
import sys
import shutil
import subprocess
sys.path.append("..")
sys.path.append("../results")
print(os.getcwd() )
from numpy import loadtxt

/home/m.giraud/auxin_sizeBuds/CPlantBox/applications/phloem_flow/resultsConsolidated


In [250]:
N1 = loadtxt('CalibPart1_1/successThreads.txt',delimiter=",")
N2 =  loadtxt('CalibPart1_2/successThreads.txt',delimiter=",")
N3 = loadtxt('CalibPart1_3/successThreads.txt',delimiter=",")
N4 = loadtxt('CalibPart1_4/successThreads.txt',delimiter=",")
successThreadsAll = np.array(np.concatenate((N1,N2,N3,N4)),dtype="int64")

N1 = N1[N1 != -1] 
N2 = N2[N2 != -1] 
N3 = N3[N3 != -1] 
N4 = N4[N4 != -1] 
successThreads = np.array(np.concatenate((N1,N2,N3,N4)),dtype="int64")
successFolder =  np.concatenate((np.full(len(N1),1),np.full(len(N2),2),np.full(len(N3),3),np.full(len(N4),4)))

endTime1 = np.full(len(successThreads),0)
Qsv = np.full(len(successThreads),0.)
MulimSucv = np.full(len(successThreads),0.)
GrRatiov = np.full(len(successThreads),0.)
nodeDv = np.full(len(successThreads),0)
CarbonCostv = np.full(len(successThreads),0.)
for n,thread in enumerate(successThreads):
    folder = successFolder[n]
    time_all_name = "CalibPart1_"+ str(folder)+"/time_all_" + str(thread)+".txt"
    data = pd.read_csv(time_all_name, header = None)
    endTime1[n] = data.loc[:,5].to_numpy( )[-1]
    time_all_name = "CalibPart1_"+ str(folder)+"/input_" + str(thread)+".txt"
    data = pd.read_csv(time_all_name, header =0)
    Qsv[n] = np.round(data["Qmax_"]*1e6)
    MulimSucv[n] = data["thresholdSuc"]
    nodeDv[n] = data["nodeD"]
    GrRatiov[n] = data["GrRatio"]
    CarbonCostv[n] = data["CarbonCost"]
    
# print(len(successThreads))

import CalibP1Database 
from importlib import reload  # Python 3.4+
CalibP1Database = reload(CalibP1Database)
    
params = CalibP1Database.toTry()
Qsvt=params['Qsv'][successThreadsAll != -1]
Qsvt = np.round(Qsv)
MulimSucvt=params['MulimSucv'][successThreadsAll != -1]
nodeDvt=np.array(params['nodeDv'][successThreadsAll != -1],dtype = "int64")
GrRatiovt= params['GrRatiov'][successThreadsAll != -1]
CarbonCostvt= params['CarbonCostv'][successThreadsAll != -1]
assert((Qsvt == Qsv).all())
assert((MulimSucvt == MulimSucv).all())
assert((GrRatiovt == GrRatiov).all())
assert((nodeDvt == nodeDv).all())
assert((CarbonCostvt == CarbonCostv).all())
print("success")

#==> new things to try? no only those where all nodes succeeded

success


In [248]:
print(set(params['nodeDv']), set(params['nodeDv'][successThreadsAll != -1]))

{3, 4, 5, 6, 7, 8} {3, 5, 6, 7, 8}


In [251]:

aa = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,CarbonCostvt],values = successThreads, aggfunc = sum)
aaFolder = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,CarbonCostvt],values = successFolder, aggfunc = sum)
aaTime = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,CarbonCostvt],values = endTime1/3600, aggfunc = sum)
aaint = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,CarbonCostvt],values = 1, aggfunc = sum)

print(max(aaint.sum(axis=0)))
#1.5 * 120 and 1.5 * 230
#0.6 * 120 and 0.6*230
bb = aa.iloc[:,np.where(aaint.sum(axis=0) == max(aaint.sum(axis=0)))[0]]
bbFolder = aaFolder.iloc[:,np.where(aaint.sum(axis=0) == max(aaint.sum(axis=0)))[0]]
bbTime = aaTime.iloc[:,np.where(aaint.sum(axis=0) == max(aaint.sum(axis=0)))[0]]
sshape = bb.shape
print(sshape,sshape[0]*sshape[1] )

6.0
(6, 7) 42


In [266]:
aa.to_csv("outAll.csv",index=True)
aaint.to_csv("outTest.csv",index=True)
bb.to_csv("out.csv",index=True)
bbFolder.to_csv("outFolder.csv",index=True)
bbTime.to_csv("outTime.csv",index=True)
bbShort = bb.iloc[:,(bbTime.max(axis=0)<2.).to_numpy()]
bbShort.to_csv("outTimeShort.csv",index=True)

In [268]:
successThreads = np.array(bbShort.to_numpy( ).flatten(),dtype="int64")
successFolder  = np.array(bbFolder.to_numpy( ).flatten(),dtype="int64")

Qsv = np.full(len(successThreads),0.)
MulimSucv = np.full(len(successThreads),0.)
GrRatiov = np.full(len(successThreads),0.)
nodeDv = np.full(len(successThreads),0)
CarbonCostv = np.full(len(successThreads),0.)
for n,thread in enumerate(successThreads):
    folder = successFolder[n]
    time_all_name = "CalibPart1_"+ str(folder)+"/time_all_" + str(thread)+".txt"
    data = pd.read_csv(time_all_name, header = None)
    time_all_name = "CalibPart1_"+ str(folder)+"/input_" + str(thread)+".txt"
    data = pd.read_csv(time_all_name, header =0)
    Qsv[n] = np.round(data["Qmax_"]*1e6)
    MulimSucv[n] = data["thresholdSuc"]
    nodeDv[n] = data["nodeD"]
    GrRatiov[n] = data["GrRatio"]
    CarbonCostv[n] = data["CarbonCost"]
    
Qsv=Qsv.reshape((-1, 1))
MulimSucv=MulimSucv.reshape((-1, 1))
nodeDv=nodeDv.reshape((-1, 1))
GrRatiov= GrRatiov.reshape((-1, 1))
CarbonCostv= CarbonCostv.reshape((-1, 1))
df = pd.DataFrame(np.concatenate([nodeDv,Qsv,MulimSucv ,GrRatiov,CarbonCostv], axis=1), columns= ["nodeD","Qs","MulimSuc" ,"GrRatio","CarbonCost"])
df.to_csv("paramsPairCalibP1.csv",index=True)

In [269]:
dfP2 = pd.read_csv('paramsPairCalibP1.csv')#.to_numpy()[:,1:]
#print(dfP2)
print(sum(dfP2.duplicated()))

0


In [270]:
#test for database
dfP2 = pd.read_csv('paramsPairCalibP1.csv')
assert(sum(dfP2.duplicated()) == 0)
kss = np.array([0.01, 0.1,0.4,0.6])
kaa = np.array([ 1. , 3.,  5.,  7.,  9., 11.])

Qs =          np.tile((dfP2["Qs"]*1e-6).to_numpy(),  len(kss)*len(kaa)) #numpy.tile(A, reps)[source]
MulimSuc =    np.tile(dfP2["MulimSuc"].to_numpy(),   len(kss)*len(kaa))
GrRatio =     np.tile(dfP2["GrRatio"].to_numpy(),    len(kss)*len(kaa))
CarbonCost =  np.tile(dfP2["CarbonCost"].to_numpy(), len(kss)*len(kaa))
nodeD =       np.tile(dfP2["nodeD"].to_numpy(),      len(kss)*len(kaa))

kss2 =        np.tile(np.repeat(kss,len(dfP2["Qs"].to_numpy())),len(kaa))
kaa2 =        np.repeat(kaa, len(dfP2["Qs"]*1e-6)*len(kss)) #numpy.repeat(a, repeats)
print(len(Qs),len(kss2), len(kaa2))

dictPara= {'Qsv' : Qs,
                'nodeDv':nodeD, 
           'MulimSucv':MulimSuc,
                'kaav':kaa2, 
           'kssv':kss2,
          'GrRatiov':GrRatio,
          'CarbonCost':CarbonCost}
df = pd.DataFrame(data=dictPara)
print(sum(df.duplicated()))
print(np.where(df.duplicated().to_numpy())[0])
print(df.iloc[df.duplicated().to_numpy(),:])
df.to_csv("forP2.csv",index=True)

1008 1008 1008
0
[]
Empty DataFrame
Columns: [Qsv, nodeDv, MulimSucv, kaav, kssv, GrRatiov, CarbonCost]
Index: []


In [274]:
Qsv= df['Qsv']
print(Qsv)
print(Qsv[2] )

0       0.00045
1       0.00045
2       0.00045
3       0.00045
4       0.00045
         ...   
1003    0.00045
1004    0.00045
1005    0.00045
1006    0.00056
1007    0.00056
Name: Qsv, Length: 1008, dtype: float64
0.00045


In [103]:
#calib P2

N1 = loadtxt('CalibPart3_1/successThreads.txt',delimiter=",")
N2 =  loadtxt('CalibPart3_2/successThreads.txt',delimiter=",")
N3 = loadtxt('CalibPart3_3/successThreads.txt',delimiter=",")

In [104]:
N2[N2 != -1] = N2[N2 != -1] +len(N1)-1
N3[N3 != -1] = N3[N3 != -1] +len(N1)+len(N2)-1
successThreads = np.concatenate((N1,N2,N3))
successThreads = np.array(successThreads[successThreads!= -1], dtype = np.int64)

In [105]:
from CalibP2Database import toTry
params = toTry()
Qsv=params['Qsv'][successThreads]
Qsv = np.round(Qsv*1e6)
MulimSucv=params['MulimSucv'][successThreads]
nodeDv=params['nodeDv'][successThreads]
GrRatiov= params['GrRatiov'][successThreads]
kssv=params['kssv'][successThreads]
kaav= params['kaav'][successThreads]

In [82]:

aa = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,kssv,kaav],values = successThreads, aggfunc = sum)
aaint = pd.crosstab(index=nodeDv, columns=[Qsv,MulimSucv ,GrRatiov,kssv,kaav],values = 1, aggfunc = sum)
#print(aaint)
print(max(aaint.sum(axis=0)))
#1.5 * 120 and 1.5 * 230
#0.6 * 120 and 0.6*230
bb = aa.iloc[:,np.where(aaint.sum(axis=0) == max(aaint.sum(axis=0)))[0]]
sshape = bb.shape
print(sshape,sshape[0]*sshape[1] )
bb.to_csv("outP2.csv",index=True)

4.0
(5, 26) 130


In [ ]:
##set new P2

Qs =np.array([230.])*1e-6
#MulimSuc =np.array([0.3,0.6,0.9,1.2,1.5])
MulimSuc =np.array([0.6,0.9,1.2])
GrRatio = np.array([3]) 
nodeD = np.array([3,4,5,6,7,8])
kss = np.array([0.01, 0.2,0.4])# np.array([0.01, 0.100, 0.2, 0.3, 0.4])
kaa = np.array([ 1. , 3.,  5.])#,  7.,  9., 11.,15,17])
maxrun = len(Qs) * len(MulimSuc) * len(nodeD) * len(GrRatio)*len(kaa)*len(kss)


Qsv,MulimSucv,kssv,kaav, nodeDv,GrRatiov = np.meshgrid(Qs,MulimSuc,kss,kaa,nodeD,GrRatio)
Qsv=Qsv.flatten()
MulimSucv=MulimSucv.flatten()
kaav=kaav.flatten()
kssv = kssv.flatten()
nodeDv=nodeDv.flatten()
GrRatiov = GrRatiov.flatten()
dictPara= {'Qsv' : Qsv,
                'nodeDv':nodeDv, 
           'MulimSucv':MulimSucv,
                'kaav':kaav, 
           'kssv':kssv,
          'GrRatiov':GrRatiov}

In [3]:
import numpy as np
Qs =np.array([200.,  340.,  450.,  560., ])*1e-6
#Qs =np.array([230.,240,250,  260,270,280,290,300,310,320,330])*1e-6
#MulimSuc =np.array([0.3,0.6,0.9,1.2,1.5])
MulimSuc =np.array([0.4,0.9,1.4])
GrRatio = np.array([3, 6, 10]) 
CarbonCost = np.array([1, 6, 10]) 
nodeD = np.array([4,8])
maxrun = len(Qs) * len(MulimSuc) * len(nodeD) * len(GrRatio)*len(CarbonCost)
print(maxrun)

216
